# Анаморфные картины

![<Здесь должно быть изображение>](../img/anamorf.jpg  "Анаморфное изображение")

Анаморфное изображение &mdash; это искажённое изображение на неровной поверхности, которое с определённой точки зрения становится ровным

Сходные вещи освещены здесь
- [Мадоннари](https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D0%B4%D0%BE%D0%BD%D0%BD%D0%B0%D1%80%D0%B8) на Википедии
- [Анаморфоз](https://ru.wikipedia.org/wiki/%D0%90%D0%BD%D0%B0%D0%BC%D0%BE%D1%80%D1%84%D0%BE%D0%B7_(%D0%B8%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%BE)) на Википедии

Существуют такие изображения пейзажа, что если поставить в определённую точку зеркальный цилиндр, пейзаж отобразится в нём в какое-то другое изображение, например, портрет человека

# Цель

Цель проекта: научиться по имеющемуся "портрету" создавать "пейзаж". Более амбициозная задача: создать редактор с окнами "Пейзаж" и "Портрет", при редактировании в одном окне меняется и другое

# Постановка геометрической задачи

Задаётся система декартовых координат. Существует цилиндр радиуса $r$ с осью $Oz$. На расстоянии $d$ от его поверхности и на высоте $h$ от плоскости $xOy$ находится точка зрения $F = (0, r + d, h)$. Проводится плоскость $\Pi: y = r$

Луч света выходит из точки $F$, проходит через точку $E = (x_E, r, z_E)$ плоскости $\Pi$, попадает в точку $C = (x_C, y_C, z_C)$ поверхности цилиндра, отражается от неё и попадает в точку $A = (x_A, y_A, 0)$ на $xOy$

Для света есть два правила: 
 - угол падения света равен углу отражения
 - падающий и отражённый лучи лежат в одной плоскости с перпендикуляром к поверхности
Для описания этих углов вводится точка $B = (0, 0, z_C)$, которая является основанием перпендикуляра из $C$ на ось цилиндра. То есть угол $ACB$ равен углу $FCB$ и точки $A, B, C, F$ лежат в одной плоскости, а $C, E, F$ на одной прямой

Высчитывать углы неудобно, поэтому используется следствие из равенства углов: $BC$ биссектриса угла $ACF$. Отметим точку $M$ как пересечение $AF$ и $BC$. В таком случае по свойству биссектрисы верно соотношение
$$\frac{AC}{FC} = \frac{AM}{FM}$$
Если опустить перпендикуляры из $AH_A$ и $FH_F$ на плоскость $\sigma: z = z_C$, то из подобия треугольников $AH_AM$ и $FH_FM$ слеует
$$\frac{AC}{FC} = \frac{AM}{FM} = \frac{AH_A}{FH_F} = \frac{z_C}{h - z_C}$$

Итак, величины $r$, $h$ и $d$ задаются как параметры. Нужно научиться выражать одну из точек $A, C, E$ через другую

## Визуализация задачи

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash_core_components as dcc
import dash_html_components as html
from dash import no_update
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash

from is_jupyterlab_running import is_jupyterlab_running
from problem import Problem, Vector, VectorRegressor
from problem.figures import Cylinder

import traceback

In [2]:
r = 100
h = 200
d = 100
n, m = 100, 100
problem = Problem(r=r, d=d, h=h)

In [3]:
cylinder = Cylinder(r=r, h=h, d=d)

xOy = go.Surface(
    x=np.outer(np.ones(n), np.linspace(-4 * r, 4 * r, m)),
    y=np.outer(np.linspace(-4 * r, 2 * r + 2 * d, n), np.ones(m)),
    z=np.outer(np.zeros(n), np.zeros(m)),
)

y_eq_r = go.Surface(
    x=np.outer(np.ones(n), np.linspace(-4 * r, 4 * r, m)),
    y=r * np.outer(np.ones(n), np.ones(m)),
    z=np.outer(np.linspace(0, h, n), np.ones(m)),
    opacity=0.7,
    colorscale=[[0, "orange"], [1, "orange"]],
)

fig_layout = dict(
    scene=dict(
        xaxis=dict(nticks=4, range=[-4 * r, 4 * r],),
        yaxis=dict(nticks=4, range=[-4 * r, 2 * r + 2 * d],),
        zaxis=dict(nticks=4, range=[-h, h],),
        aspectratio=dict(x=1, y=1, z=1),
    ),
    height=1000,
)

surface_labels = ["cylinder", "xOy", "y=r", "ray"]

In [4]:
# without ray

fig = go.Figure(
    data=[cylinder.surface(n, m), xOy, y_eq_r], layout=fig_layout
)

app = JupyterDash(__name__)
app.layout = html.Div(
    [html.H1("Geometry "),
     dcc.Graph(id="graph", figure=fig)]
)

app.run_server(mode="jupyterlab" if is_jupyterlab_running()
               else "inline", port=8050 if is_jupyterlab_running() else 8051)

In [5]:
# with ray

start_c_vector = problem.get_c(x=0, z=h / 2)
start_ray = problem.get_ray(c=start_c_vector)

fig = go.Figure(
    data=[cylinder.surface(n, m), xOy, y_eq_r, start_ray.trace], layout=fig_layout
)

app = JupyterDash(__name__)
app.layout = html.Div(
    [html.H1("Geometry "),
     dcc.Graph(id="graph", figure=fig, animate=True, animation_options={"frame":{"redraw":True}}),
     html.P(id="stdout"), html.P(id="stderr", style={"color": "#f00"})]
)


@app.callback(
    Output("graph", "figure"),
    Output("stdout", "children"),
    Output("stderr", "children"),
    Input("graph", "clickData"),
    State("graph", "figure"),
)
def update_ray(click_data, figure):
    if click_data is None:
        raise PreventUpdate
    try:
        figure["layout"]["scene"]
        data = click_data["points"][0]
        point = Vector(data["x"], data["y"], data["z"])
        label = surface_labels[data["curveNumber"]]
        if label == "cylinder":
            ray = problem.get_ray(c=point)
        elif label == "xOy":
            ray = problem.get_ray(a=point)
        elif label == "y=r":
            ray = problem.get_ray(e=point)
        else:
            return no_update, str(data), "another surface"
        if ray.c.y < problem.r ** 2 / (problem.r + problem.d):
            return no_update, str(data), "point C is not seen"
        figure["data"][3] = ray.trace
        return figure, html.Div([html.P(str(data)),
                                 html.P(f"a = {ray.a}"),
                                 html.P(f"c = {ray.c}"),
                                 html.P(f"e = {ray.e}"),
                                 html.P(f"f = {ray.f}")]), no_update
    except BaseException as e:
        return no_update, str(data), html.Div([
            html.P(str(data)),
            html.Pre(
                ''.join(traceback.TracebackException.from_exception(e).format())),
            html.P('base exception')
        ])


app.run_server(mode="jupyterlab" if is_jupyterlab_running()
               else "inline", port=8050 if is_jupyterlab_running() else 8051)

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [<bound method Ray.trace of Ray(a=Vector(x=0.0, y=200.0, z=0), c=Vector(x=0, y=100.0, z=100.0), e=Vector(x=0.0, y=0, z=100.0), f=Vector(x=0, y=200, z=200))>]

    The 'data' property is a tuple of trace instances
    that may be specified as:
      - A list or tuple of trace instances
        (e.g. [Scatter(...), Bar(...)])
      - A single trace instance
        (e.g. Scatter(...), Bar(...), etc.)
      - A list or tuple of dicts of string/value properties where:
        - The 'type' property specifies the trace type
            One of: ['area', 'bar', 'barpolar', 'box',
                     'candlestick', 'carpet', 'choropleth',
                     'choroplethmapbox', 'cone', 'contour',
                     'contourcarpet', 'densitymapbox', 'funnel',
                     'funnelarea', 'heatmap', 'heatmapgl',
                     'histogram', 'histogram2d',
                     

# Формальные условия

$$r > 0, h > 0, d > 0$$
$$x_A^2 + y_A^2 > r^2$$

$$x_C^2 + y_C^2 = r^2$$
$$0 \le z_C < h$$

$$\begin{vmatrix} 0 & x_A & x_C\\ r+d & y_A & y_C \\ h - z_C & - z_C & 0 \end{vmatrix} = 0$$

$$\frac{\sqrt{(x_C - x_A)^2 + (y_C - y_A)^2 + z_C^2}}{\sqrt{x_C^2 + (r + d - y_C)^2 + (h - z_C)^2}} = \frac{z_C}{h - z_C}$$

$$\frac{x_C - x_E}{x_C} = \frac{r - y_C}{r + d - y_C} = \frac{z_E - z_C}{h - z_C}$$

# Известно C, нужно найти A

Преобразование определителя (7)
$$\begin{vmatrix} 0 & x_A & x_C\\ r+d & y_A & y_C \\ h - z_C & - z_C & 0 \end{vmatrix} = 0$$

$$0\cdot y_A\cdot 0 + x_A\cdot y_C\cdot(h - z_C) + x_C\cdot(r+d)\cdot(-z_C) - x_C\cdot y_A\cdot(h-z_C) - x_A\cdot(r+d)\cdot 0 - 0\cdot y_C\cdot(-z_C) = 0$$

$$x_A\cdot y_C\cdot(h - z_C) + x_C\cdot(r+d)\cdot(-z_C) - x_C\cdot y_A\cdot(h-z_C) = 0$$

$$y_A = \frac{x_A\cdot y_C\cdot(h - z_C) + x_C\cdot(r+d)\cdot(-z_C)}{x_C\cdot(h-z_C)}  $$

Преобразование отношения длин (8)

$$\frac{\sqrt{(x_C - x_A)^2 + (y_C - y_A)^2 + z_C^2}}{\sqrt{x_C^2 + (r + d - y_C)^2 + (h - z_C)^2}} = \frac{z_C}{h - z_C}$$

$$\frac{(x_C - x_A)^2 + (y_C - y_A)^2 + z_C^2}{x_C^2 + (r + d - y_C)^2 + (h - z_C)^2} = \frac{z_C^2}{(h - z_C)^2}$$

преобразование $\frac{a+b}{c+d} = \frac{a}{c} \Leftrightarrow \frac{a}{c} = \frac{b}{d}$

$$\frac{(x_C - x_A)^2 + (y_C - y_A)^2}{x_C^2 + (r + d - y_C)^2} = \frac{z_C^2}{(h - z_C)^2}$$                       

$$(x_C - x_A)^2 + (y_C - y_A)^2 = \frac{z_C^2}{(h - z_C)^2}(x_C^2 + (r + d - y_C)^2)$$

Подстановка формулы (13) в формулу (17)

$$(x_C - x_A)^2 + \left(y_C - \frac{x_A\cdot y_C\cdot(h - z_C) + x_C\cdot(r+d)\cdot(-z_C)}{x_C\cdot(h-z_C)}\right)^2 = \frac{z_C^2}{(h - z_C)^2}(x_C^2 + (r + d - y_C)^2)$$

Некоторые преобразования

$$(x_C - x_A)^2 + \left(\frac{y_C}{x_C}(x_C - x_A) + \frac{z_C}{h-z_C}(r+d)\right)^2 = \frac{z_C^2}{(h - z_C)^2}(x_C^2 + (r + d)^2 - 2y_C(r + d) + y_C^2)$$

$$(x_C - x_A)^2 + \frac{y_C^2}{x_C^2}(x_C - x_A)^2 + 2\frac{y_C\cdot z_C}{x_C(h - z_C)}(x_C - x_A)(r + d) + \frac{z_C^2}{(h - z_C)^2}(r + d)^2 = \frac{z_C^2}{(h - z_C)^2}(x_C^2 + y_C^2) + \frac{z_C^2}{(h - z_C)^2}(r + d)^2 - 2\frac{z_C^2}{(h - z_C)^2}y_C(r + d)$$

$$\frac{r^2}{x_C^2}(x_A^2 + x_C^2 - 2x_A\cdot x_C)^2 + 2\frac{y_C\cdot z_C}{x_C(h - z_C)}(x_C - x_A)(r + d) = \frac{z_C^2}{(h - z_C)^2}r^2 - 2\frac{z_C^2}{(h - z_C)^2}y_C(r + d)$$

$$\frac{r^2}{x_C^2}(x_A^2 + x_C^2 - 2x_A\cdot x_C) + 2\frac{y_C\cdot z_C}{x_C(h - z_C)}(x_C - x_A)(r + d) = \frac{z_C^2}{(h - z_C)^2}r^2 - 2\frac{z_C^2}{(h - z_C)^2}y_C(r + d)$$

Получается квадратный трёхчлен относительно переменной $x_A$. Приведение трёхчлена к стандартному виду

$$\frac{r^2}{x_C^2}x_A^2 + x_A\left(\frac{r^2}{x_C^2}(-2x_C) - 2\frac{y_C\cdot z_C}{x_C(h - z_C)}(r + d)\right) + \left(r^2 + 2\frac{y_C\cdot z_C}{h - z_C}(r + d) + 2\frac{z_C^2}{(h - z_C)^2}y_C(r+d) - \frac{z_C^2}{(h - z_C)^2}r^2\right) = 0$$

$$\frac{r^2}{x_C^2}x_A^2 - 2x_A\left(\frac{r^2}{x_C} + \frac{y_C\cdot z_C}{x_C(h - z_C)}(r + d)\right) + \left(r^2 + 2\frac{y_C\cdot z_C\cdot h}{(h - z_C)^2}(r+d) - \frac{z_C^2}{(h - z_C)^2}r^2\right) = 0$$

Вычисление дискриминанта

$$D = 4\left(\frac{r^2}{x_C} + \frac{y_C\cdot z_C}{x_C(h - z_C)}(r + d)\right)^2 - 4\frac{r^2}{x_C^2} \left(r^2 + 2\frac{y_C\cdot z_C\cdot h}{(h - z_C)^2}(r+d) - \frac{z_C^2}{(h - z_C)^2}r^2\right)$$

$$D = 4\left(\frac{r^4}{x_C^2} + \frac{y_C^2 z_C^2}{x_C^2(h - z_C)^2}(r + d)^2 + 2\frac{r^2y_C\cdot z_C}{x_C^2(h - z_C)}(r + d) \right) - 4\left(\frac{r^4}{x_C^2} + 2\frac{r^2 y_C\cdot z_C\cdot h}{x_C^2(h - z_C)^2}(r+d) - \frac{r^2 z_C^2}{x_C^2(h - z_C)^2}r^2\right)$$

$$D = 4\left(\frac{y_C^2 z_C^2}{x_C^2(h - z_C)^2}(r + d)^2 + 2\frac{r^2y_C\cdot z_C}{x_C^2(h - z_C)}(r + d) \right) - 4\left(2\frac{r^2 y_C\cdot z_C\cdot h}{x_C^2(h - z_C)^2}(r+d) - \frac{r^2 z_C^2}{x_C^2(h - z_C)^2}r^2\right)$$

$$D = 4\left(\frac{y_C^2 z_C^2}{x_C^2(h - z_C)^2}(r + d)^2 - 2\frac{r^2y_C\cdot z_C^2}{x_C^2(h - z_C)^2}(r + d) + \frac{r^2 z_C^2}{x_C^2(h - z_C)^2}r^2\right)$$

$$D = 4\frac{z_C^2}{x_C^2(h - z_C)^2}(y_C^2(r + d)^2 - 2r^2y_C(r + d) + r^4)$$

$$D = 4\frac{z_C^2}{x_C^2(h - z_C)^2}(y_C(r + d) - r^2)^2$$

$$x_A = \frac{\left(\frac{r^2}{x_C} + \frac{z_C}{x_C(h - z_C)}y_C(r + d)\right) \pm \frac{z_C}{x_C(h - z_C)}(y_C(r + d) - r^2)}{\frac{r^2}{x_C^2}}$$

Получаются два решения
$$x_{A_1} = \frac{h\cdot x_C}{h - z_C}$$
или
$$x_{A_2} = \frac{h - 2z_C}{h - z_C}x_C + 2\frac{x_C\cdot y_C\cdot z_C}{r^2 (h - z_C)}(r + d)$$

Подставляем формулу (32) в формулу (14)

$$y_{A_1} = \frac{h \cdot y_C - z_C(r+d)}{(h-z_C)}$$

Подставляем формулу (33) в формулу (14)

$$y_{A_2} = \frac{h - 2z_C}{h - z_C}y_C + 2\frac{y_C^2 z_C}{r^2(h - z_C)}(r + d) - \frac{z_C}{h - z_C}(r + d)$$

$$y_{A_2} = \frac{h - 2z_C}{h - z_C}y_C + \frac{(y_C^2 - x_C^2) z_C}{r^2(h - z_C)}(r + d)$$

## Анализ результатов

Были применены два требования: точки $F, A, B, C$ лежат в одной плоскости, и расстояния от точек $A$ и $F$ до $C$ пропорциональны длинам перпендикуляров из $AH_A$ и $FH_F$ на плоскость $z = z_C$. Получились две точки, потому что под эти требования подходит ещё и точка пересечения прямой $FC$ и плоскости $xOy$, но она не подходит под исходные требования. Теперь нужно определить, какая из двух точек лишняя

Рассмотрим задачу при значениях $r=1, h=2, d=1$, $x_C = 0, y_C = 1, z_C = 1$. Тогда $x_{A_1} = 0, x_{A_2} = 0, y_{A_1} = 0, y_{A_2} = 2$. Точка $A_1$ не соответствует неравенству (4). Следовательно, искомой является точка $A_2$, $A\equiv A_2$

$$x_A = \frac{h - 2z_C}{h - z_C}x_C + 2\frac{x_C\cdot y_C\cdot z_C}{r^2 (h - z_C)}(r + d)$$

$$y_A = \frac{h - 2z_C}{h - z_C}y_C + \frac{(y_C^2 - x_C^2) z_C}{r^2(h - z_C)}(r + d)$$

# Известно C, нужно найти E

Выразим $x_E$ и $z_E$ из (9) и (5)

$$\frac{x_C - x_E}{x_C} = \frac{r - y_C}{r + d - y_C} = \frac{z_E - z_C}{h - z_C}$$

$$\frac{x_C - x_E}{x_C} = \frac{r - y_C}{r + d - y_C}$$

$$x_E(r+d-y_C) = x_C d$$

$$x_E = \frac{x_Cd}{r+d-y_C}$$

$$\frac{r - y_C}{r + d - y_C} = \frac{z_E - z_C}{h - z_C}$$

$$rh-y_Ch+z_Cd = z_E(r+d-y_C)$$

$$z_E=\frac{(r-y_C)h+z_Cd}{r+d-y_C}$$
